In [1]:
import random
from deap import base, creator, tools, algorithms
import pandas as pd
import numpy as np
from functools import partial


In [2]:
path = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/供应商综合评价排名.xlsx"
df = pd.read_excel(path, sheet_name='Sheet1')

# 输出到Excel文件
path1 = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/由大到小综合评价的索引.xlsx"
df1 = pd.read_excel(path1, sheet_name='Sheet1')

path2 = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/附件1 近5年402家供应商的相关数据.xlsx"
df2 = pd.read_excel(path2, sheet_name='供应商的供货量（m³）')

data0 = df2.iloc[:, :].values


In [3]:
index = df1.iloc[:50, :].values.tolist()
# 将所有子列表合并为一个列表
index = np.array([item for sublist in index for item in sublist])


In [17]:
df2.iloc[index, :].values
sales = np.array([item for item in df2.iloc[index, :].values])
sales


array([['S229', 'A', 2006, ..., 1899, 1574, 1666],
       ['S361', 'C', 910, ..., 2221, 1015, 1012],
       ['S108', 'B', 556, ..., 636, 755, 698],
       ...,
       ['S208', 'A', 0, ..., 0, 0, 0],
       ['S154', 'A', 0, ..., 244, 153, 83],
       ['S189', 'A', 71, ..., 0, 0, 0]], dtype=object)

In [18]:
# 创建DataFrame保存结果
result_df = pd.DataFrame(sales # 只取供货量数据（去掉前两列）
)

# 输出到Excel文件
output_path = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/前50家商家供货量.xlsx"
result_df.to_excel(output_path, index=False)

print(f"由大到小综合评价的索引已保存到 {output_path}")

由大到小综合评价的索引已保存到 C:/Users/海棠微雨/Desktop/cumcm2021c/C/前50家商家供货量.xlsx


In [69]:
# 仓库初始库存
initial_stock = 55800  # 立方米

# 根据索引提取前50个供应商的数据
data = df2.iloc[index, 2:].values  # 只取供货量数据（去掉前两列）

# 设置企业的需求参数
weekly_demand = 2.82 * 10**4
target_supply = 2 * weekly_demand  # 两周的需求

# 创建适应度和个体类
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# 初始化函数：随机选择n个供应商
def initIndividual(icls, data, n_suppliers):
    idx = random.sample(range(data.shape[0]), n_suppliers)
    individual = [data[i].sum() for i in idx]
    return icls(individual)

# 适应度函数：计算供应商总供应量与目标供货量（目标值减去初始库存）的差值（最小化差值）
def evalFunc(individual):
    total_supply = sum(individual) + initial_stock
    return abs(total_supply - target_supply),  # 返回一个元组，适应度是差值的绝对值

# 设置工具箱
toolbox = base.Toolbox()
toolbox.register("evaluate", evalFunc)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# 主函数
def main():
    for n_suppliers in range(1, 51):  # 从1个供应商开始，逐步增加
        # 如果已经注册，取消注册
        if hasattr(toolbox, "individual"):
            delattr(toolbox, "individual")
        
        # 注册新的individual生成函数
        toolbox.register("individual", partial(initIndividual, icls=creator.Individual, data=data, n_suppliers=n_suppliers))
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        
        pop = toolbox.population(n=300)  # 设置种群规模
        cxpb, mutpb, ngen = 0.7, 0.3, 100  # 交叉概率、变异概率和代数
        pop, log = algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, 
                                       stats=None, halloffame=None, verbose=False)
        
        best_individual = tools.selBest(pop, 1)[0]
        best_total_supply = sum(best_individual) + initial_stock
        
        if abs(best_total_supply - target_supply) <= 1:  # 满足条件的误差范围（单位为立方米）
            print(f"Minimum suppliers needed: {n_suppliers}")
            print(f"Best individual: {best_individual}, with total supply {best_total_supply} including initial stock {initial_stock}")
            break

if __name__ == "__main__":
    main()

Minimum suppliers needed: 1
Best individual: [600.0000000000003], with total supply 56400.0 including initial stock 55800
